In [15]:
# 한 번에 모든 라이브러리 설치 (가장 간단)
!pip install pandas numpy matplotlib seaborn scipy scikit-learn tqdm lightgbm xgboost catboost optuna eli5 joblib

/bin/bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
Error processing line 1 of /opt/conda/envs/stages/lib/python3.10/site-packages/distutils-precedence.pth:

  Traceback (most recent call last):
    File "/opt/conda/envs/stages/lib/python3.10/site.py", line 186, in addpackage
      exec(line)
    File "<string>", line 1, in <module>
  ModuleNotFoundError: No module named '_distutils_hack'

Remainder of file ignored
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 35.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 MB 79.9 MB/s eta 0:00:00a 0:00:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 71.2 MB/s eta 0:00:0000:0100:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 61.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.5/322.5 MB 101.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.8/9.8 MB 76.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 584.4/584.4 kB

In [23]:
# visualization
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
fe = fm.FontEntry(
    fname=r'/usr/share/fonts/truetype/nanum/NanumGothic.ttf', # ttf 파일이 저장되어 있는 경로
    name='NanumBarunGothic')                        # 이 폰트의 원하는 이름 설정
fm.fontManager.ttflist.insert(0, fe)              # Matplotlib에 폰트 추가
plt.rcParams.update({'font.size': 10, 'font.family': 'NanumBarunGothic'}) # 폰트 설정
plt.rc('font', family='NanumBarunGothic')
import seaborn as sns

# utils
import re
import pandas as pd
import numpy as np
from tqdm import tqdm
import pickle
import warnings;warnings.filterwarnings('ignore')


# 고성능 모델링 라이브러리
import lightgbm as lgb
import xgboost as xgb
from catboost import CatBoostRegressor
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor
from sklearn.linear_model import Ridge, Lasso
from sklearn.preprocessing import LabelEncoder, StandardScaler, RobustScaler
from sklearn.model_selection import KFold, TimeSeriesSplit, train_test_split
from sklearn.metrics import mean_squared_error
import optuna
from scipy import stats


import eli5
from eli5.sklearn import PermutationImportance

In [12]:
def load_and_preprocess_data(train_path, test_path):
    """향상된 데이터 로딩 및 전처리"""

    print("📊 데이터 로딩 중...")
    dt = pd.read_csv(train_path)
    dt_test = pd.read_csv(test_path)

    # 데이터 기본 정보
    print(f"Train 데이터: {dt.shape}")
    print(f"Test 데이터: {dt_test.shape}")
    print(f"타겟 분포: 평균 {dt['target'].mean():.0f}만원, 표준편차 {dt['target'].std():.0f}만원")

    # 타겟 변수 로그 변환 (RMSE 개선에 효과적)
    dt['target_log'] = np.log1p(dt['target'])

    # train/test 구분
    dt['is_test'] = 0
    dt_test['is_test'] = 1
    dt_test['target'] = np.nan
    dt_test['target_log'] = np.nan

    concat = pd.concat([dt, dt_test], ignore_index=True)

    return concat

In [4]:
def advanced_missing_value_treatment(df):
    df['등기신청일자'] = df['등기신청일자'].replace([' ', '', '-'], np.nan)
    df['거래유형'] = df['거래유형'].replace(['-', '', ' '], np.nan)
    df['중개사소재지'] = df['중개사소재지'].replace(['-', '', ' '], np.nan)

    missing_ratio = df.isnull().sum() / len(df)
    high_missing_cols = missing_ratio[missing_ratio > 0.9].index.tolist()

    print(f"결측치 90% 이상인 컬럼 {len(high_missing_cols)}개 제거: {high_missing_cols}")

    df = df.drop(columns=high_missing_cols)

    # 지역별, 시점별 그룹 평균으로 결측치 대체 (더 정교한 방법)
    for col in df.select_dtypes(include=[np.number]).columns:
        if df[col].isnull().sum() > 0:
            # 구별, 계약년월별 평균으로 대체
            if '구' in df.columns and '계약년월' in df.columns:
                df[col] = df.groupby(['구', '계약년월'])[col].transform(
                    lambda x: x.fillna(x.median())
                )
            df[col] = df[col].fillna(df[col].median())

    # 범주형 변수 결측치 처리
    for col in df.select_dtypes(include=['object']).columns:
        if df[col].isnull().sum() > 0:
            df[col] = df[col].fillna('UNKNOWN')

    return df

In [5]:
advanced_missing_value_treatment(dt)
advanced_missing_value_treatment(dt_test)

결측치 90% 이상인 컬럼 7개 제거: ['해제사유발생일', '등기신청일자', '거래유형', '중개사소재지', '단지소개기존clob', 'k-135㎡초과', 'k-등록일자']
결측치 90% 이상인 컬럼 6개 제거: ['해제사유발생일', '단지소개기존clob', 'k-135㎡초과', 'k-등록일자', 'target', 'target_log']


,시군구,번지,본번,부번,아파트명,전용면적(㎡),계약년월,계약일,층,건축년도,...,건축면적,주차대수,기타/의무/임대/임의=1/2/3/4,단지승인일,사용허가여부,관리비 업로드,좌표X,좌표Y,단지신청일,is_test
0,서울특별시 강남구 개포동,658-1,658.0,1.0,개포6차우성,79.9700,202307,26,5,1987,...,4858.00,262.0,임의,2022-11-17 13:00:29.0,Y,N,127.057210,37.476763,2022-11-17 10:19:06.0,1
1,서울특별시 강남구 개포동,651-1,651.0,1.0,개포더샵트리에,108.2017,202308,15,10,2021,...,2724.46,305.0,의무,2022-02-23 13:01:10.0,Y,N,127.056394,37.484892,2022-02-23 11:05:05.0,1
2,서울특별시 강남구 개포동,652,652.0,0.0,개포우성3차,161.0000,202307,28,15,1984,...,61064.24,419.0,의무,1984-12-22 00:00:00.0,Y,N,127.055990,37.483894,2013-03-07 09:46:28.0,1
3,서울특별시 강남구 개포동,652,652.0,0.0,개포우성3차,133.4600,202308,10,14,1984,...,61064.24,419.0,의무,1984-12-22 00:00:00.0,Y,N,127.055990,37.483894,2013-03-07 09:46:28.0,1
4,서울특별시 강남구 개포동,652,652.0,0.0,개포우성3차,104.4300,202308,18,6,1984,...,61064.24,419.0,의무,1984-12-22 00:00:00.0,Y,N,127.055990,37.483894,2013-03-07 09:46:28.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9267,서울특별시 중랑구 신내동,816,816.0,0.0,신내우디안1단지,84.6500,202307,19,13,2014,...,14171.00,1568.0,의무,2015-09-09 15:30:27.0,Y,N,127.106720,37.618870,2014-09-01 13:05:03.0,1
9268,서울특별시 중랑구 신내동,816,816.0,0.0,신내우디안1단지,84.6200,202307,25,12,2014,...,14171.00,1568.0,의무,2015-09-09 15:30:27.0,Y,N,127.106720,37.618870,2014-09-01 13:05:03.0,1
9269,서울특별시 중랑구 신내동,816,816.0,0.0,신내우디안1단지,101.6500,202308,27,12,2014,...,14171.00,1568.0,의무,2015-09-09 15:30:27.0,Y,N,127.106720,37.618870,2014-09-01 13:05:03.0,1
9270,서울특별시 중랑구 신내동,816,816.0,0.0,신내우디안1단지,84.9400,202309,2,18,2014,...,14171.00,1568.0,의무,2015-09-09 15:30:27.0,Y,N,127.106720,37.618870,2014-09-01 13:05:03.0,1


In [6]:
def advanced_feature_engineering(df):
    """고도화된 피처 엔지니어링"""

    print("🎯 고급 피처 엔지니어링 중...")

    # 기본 주소 파싱
    if '시군구' in df.columns:
        df['시'] = df['시군구'].str.split().str[0]
        df['구'] = df['시군구'].str.split().str[1]
        df['동'] = df['시군구'].str.split().str[2]
        df = df.drop('시군구', axis=1)

    # 시간 피처
    if '계약년월' in df.columns:
        df['계약년월'] = df['계약년월'].astype(str)
        df['계약년'] = df['계약년월'].str[:4].astype(int)
        df['계약월'] = df['계약년월'].str[4:].astype(int)
        df['계약분기'] = ((df['계약월'] - 1) // 3 + 1)
        df['봄여름여부'] = (df['계약월'].isin([3,4,5,6])).astype(int)  # 이사철

    # 건축 관련 피처
    if '건축년도' in df.columns:
        current_year = 2023
        df['건축경과년수'] = current_year - df['건축년도']
        df['신축여부'] = (df['건축경과년수'] <= 5).astype(int)
        df['재건축대상여부'] = (df['건축경과년수'] >= 30).astype(int)

        # 건축년도 구간화
        df['건축년대'] = (df['건축년도'] // 10) * 10

    # 면적 관련 고급 피처
    if '전용면적' in df.columns:
        df['평수'] = df['전용면적'] / 3.3058  # 평 단위 변환
        df['면적구간'] = pd.cut(df['전용면적'],
                              bins=[0, 60, 85, 135, 200, float('inf')],
                              labels=['소형', '중소형', '중대형', '대형', '초대형'])

    # 층수 관련 고급 피처
    if '층' in df.columns and '최고층' in df.columns:
        df['층비율'] = df['층'] / (df['최고층'] + 1)  # 전체 층수 대비 해당 층 비율
        df['저층여부'] = (df['층비율'] <= 0.3).astype(int)
        df['중층여부'] = ((df['층비율'] > 0.3) & (df['층비율'] <= 0.7)).astype(int)
        df['고층여부'] = (df['층비율'] > 0.7).astype(int)
        df['최고층여부'] = (df['층'] == df['최고층']).astype(int)
        df['1층여부'] = (df['층'] == 1).astype(int)

    # 지역 프리미엄 피처 (도메인 지식 활용)
    if '구' in df.columns:
        # 강남권 정의 (실제 부동산 시장 기준)
        강남권 = ['강남구', '서초구', '송파구']
        서북권 = ['종로구', '중구', '용산구', '성동구', '마포구', '영등포구']
        강북권 = ['강북구', '성북구', '도봉구', '노원구']
        서남권 = ['관악구', '동작구', '금천구', '구로구', '양천구', '강서구']
        동북권 = ['광진구', '동대문구', '중랑구']
        동남권 = ['강동구']

        df['강남권여부'] = df['구'].isin(강남권).astype(int)
        df['서북권여부'] = df['구'].isin(서북권).astype(int)
        df['권역'] = df['구'].map(lambda x:
            '강남권' if x in 강남권 else
            '서북권' if x in 서북권 else
            '강북권' if x in 강북권 else
            '서남권' if x in 서남권 else
            '동북권' if x in 동북권 else
            '동남권' if x in 동남권 else '기타'
        )

        # 학군 프리미엄 (8학군 등)
        명문학군 = ['강남구', '서초구', '송파구', '양천구']  # 목동 포함
        df['명문학군여부'] = df['구'].isin(명문학군).astype(int)

    # 아파트명에서 브랜드 추출
    if '아파트명' in df.columns:
        브랜드키워드 = ['래미안', '푸르지오', '힐스테이트', '자이', '아크로',
                     '파크리오', '센트럴', 'SK뷰', '더샵', '스타클래스',
                     '아이파크', '트럼프', '롯데캐슬']

        for 브랜드 in 브랜드키워드:
            df[f'{브랜드}_여부'] = df['아파트명'].str.contains(브랜드, na=False).astype(int)

        df['브랜드아파트여부'] = df[[f'{브랜드}_여부' for 브랜드 in 브랜드키워드]].sum(axis=1) > 0
        df['브랜드아파트여부'] = df['브랜드아파트여부'].astype(int)

    return df

In [7]:
advanced_feature_engineering(dt)
advanced_feature_engineering(dt_test)

🎯 고급 피처 엔지니어링 중...
🎯 고급 피처 엔지니어링 중...


,번지,본번,부번,아파트명,전용면적(㎡),계약년월,계약일,층,건축년도,도로명,...,아크로_여부,파크리오_여부,센트럴_여부,SK뷰_여부,더샵_여부,스타클래스_여부,아이파크_여부,트럼프_여부,롯데캐슬_여부,브랜드아파트여부
0,658-1,658.0,1.0,개포6차우성,79.9700,202307,26,5,1987,언주로 3,...,0,0,0,0,0,0,0,0,0,0
1,651-1,651.0,1.0,개포더샵트리에,108.2017,202308,15,10,2021,개포로 311,...,0,0,0,0,1,0,0,0,0,1
2,652,652.0,0.0,개포우성3차,161.0000,202307,28,15,1984,개포로 307,...,0,0,0,0,0,0,0,0,0,0
3,652,652.0,0.0,개포우성3차,133.4600,202308,10,14,1984,개포로 307,...,0,0,0,0,0,0,0,0,0,0
4,652,652.0,0.0,개포우성3차,104.4300,202308,18,6,1984,개포로 307,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9267,816,816.0,0.0,신내우디안1단지,84.6500,202307,19,13,2014,신내역로1길 85,...,0,0,0,0,0,0,0,0,0,0
9268,816,816.0,0.0,신내우디안1단지,84.6200,202307,25,12,2014,신내역로1길 85,...,0,0,0,0,0,0,0,0,0,0
9269,816,816.0,0.0,신내우디안1단지,101.6500,202308,27,12,2014,신내역로1길 85,...,0,0,0,0,0,0,0,0,0,0
9270,816,816.0,0.0,신내우디안1단지,84.9400,202309,2,18,2014,신내역로1길 85,...,0,0,0,0,0,0,0,0,0,0


In [8]:
def create_aggregation_features(df):
    """집계 피처 생성 - RMSE 개선의 핵심"""

    print("📈 집계 피처 생성 중...")

    # 훈련 데이터만으로 집계 통계 계산 (리키지 방지)
    train_data = df[df['is_test'] == 0].copy()

    # 구별 집계 피처
    if '구' in df.columns and 'target' in df.columns:
        구별_통계 = train_data.groupby('구')['target'].agg([
            'mean', 'median', 'std', 'min', 'max', 'count'
        ]).add_prefix('구별_target_')

        df = df.merge(구별_통계, left_on='구', right_index=True, how='left')

    # 동별 집계 피처
    if '동' in df.columns and 'target' in df.columns:
        동별_통계 = train_data.groupby('동')['target'].agg([
            'mean', 'median', 'count'
        ]).add_prefix('동별_target_')

        df = df.merge(동별_통계, left_on='동', right_index=True, how='left')

    # 아파트별 집계 피처
    if '아파트명' in df.columns and 'target' in df.columns:
        아파트별_통계 = train_data.groupby('아파트명')['target'].agg([
            'mean', 'median', 'count'
        ]).add_prefix('아파트별_target_')

        df = df.merge(아파트별_통계, left_on='아파트명', right_index=True, how='left')

    # 면적구간별 집계 피처
    if '면적구간' in df.columns and 'target' in df.columns:
        면적구간별_통계 = train_data.groupby('면적구간')['target'].agg([
            'mean', 'median', 'std'
        ]).add_prefix('면적구간별_target_')

        df = df.merge(면적구간별_통계, left_on='면적구간', right_index=True, how='left')

    # 시점별 트렌드 피처
    if '계약년월' in df.columns and 'target' in df.columns:
        시점별_통계 = train_data.groupby('계약년월')['target'].agg([
            'mean', 'count'
        ]).add_prefix('시점별_target_')

        df = df.merge(시점별_통계, left_on='계약년월', right_index=True, how='left')

    # 복합 집계 피처 (구 + 면적구간)
    if all(col in df.columns for col in ['구', '면적구간', 'target']):
        복합_통계 = train_data.groupby(['구', '면적구간'])['target'].agg([
            'mean', 'count'
        ]).add_prefix('구_면적구간별_target_')

        df = df.merge(복합_통계, left_on=['구', '면적구간'], right_index=True, how='left')

    return df


In [9]:
def advanced_outlier_treatment(df):
    """고도화된 이상치 처리"""

    print("🔍 이상치 처리 중...")

    train_data = df[df['is_test'] == 0].copy()
    test_data = df[df['is_test'] == 1].copy()

    # 타겟 변수 이상치 제거 (Z-score 기준)
    if 'target' in train_data.columns:
        z_scores = np.abs(stats.zscore(train_data['target']))
        train_data = train_data[z_scores < 3]  # 3 시그마 밖 제거

        print(f"타겟 이상치 제거 후: {len(train_data)}건")

    # 전용면적 이상치 제거 (IQR 기준, 더 엄격하게)
    if '전용면적' in train_data.columns:
        Q1 = train_data['전용면적'].quantile(0.25)
        Q3 = train_data['전용면적'].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR

        train_data = train_data[
            (train_data['전용면적'] >= lower_bound) &
            (train_data['전용면적'] <= upper_bound)
        ]

        print(f"전용면적 이상치 제거 후: {len(train_data)}건")

    return pd.concat([train_data, test_data], ignore_index=True)

In [20]:
def clean_feature_names(df):
    """피처명에서 특수문자 제거 및 정리"""

    print("컬럼명 정리 중...")

    # 새로운 컬럼명 매핑
    column_mapping = {}

    for col in df.columns:
        # 원본 컬럼명 저장
        original_name = col

        # 1. 한글을 영문으로 변환 (주요 컬럼들)
        name_mapping = {
            '시군구': 'district',
            '구': 'gu',
            '동': 'dong',
            '번지': 'street_num',
            '본번': 'main_num',
            '부번': 'sub_num',
            '아파트명': 'apt_name',
            '전용면적': 'area',
            '계약년월': 'contract_ym',
            '계약년': 'contract_year',
            '계약월': 'contract_month',
            '계약분기': 'contract_quarter',
            '계약일': 'contract_day',
            '층': 'floor',
            '건축년도': 'build_year',
            '건축경과년수': 'building_age',
            '최고층': 'max_floor',
            '최고층X': 'coord_x',
            '최고층Y': 'coord_y',
            '신축여부': 'is_new',
            '재건축대상여부': 'is_redevelop_target',
            '강남권여부': 'is_gangnam',
            '명문학군여부': 'is_good_school',
            '브랜드아파트여부': 'is_brand_apt',
            '면적구간': 'area_group',
            '층비율': 'floor_ratio',
            '저층여부': 'is_low_floor',
            '중층여부': 'is_mid_floor',
            '고층여부': 'is_high_floor',
            '평수': 'pyeong',
            '권역': 'region',
            '건축년대': 'build_decade',
            'target': 'target',
            'target_log': 'target_log',
            'is_test': 'is_test'
        }

        # 매핑된 이름이 있으면 사용
        if col in name_mapping:
            new_name = name_mapping[col]
        else:
            # 2. 한글 제거하고 영문/숫자만 남기기
            new_name = re.sub(r'[가-힣]', '', col)

            # 3. 특수문자를 언더스코어로 변환
            new_name = re.sub(r'[^\w]', '_', new_name)

            # 4. 연속된 언더스코어 제거
            new_name = re.sub(r'_+', '_', new_name)

            # 5. 앞뒤 언더스코어 제거
            new_name = new_name.strip('_')

            # 6. 빈 문자열이면 기본값 사용
            if not new_name or new_name.isdigit():
                new_name = f'feature_{list(df.columns).index(col)}'

            # 7. 숫자로 시작하면 앞에 문자 추가
            if new_name[0].isdigit():
                new_name = 'f_' + new_name

        column_mapping[original_name] = new_name

    # 중복 방지
    used_names = set()
    final_mapping = {}

    for original, new in column_mapping.items():
        if new in used_names:
            counter = 1
            while f"{new}_{counter}" in used_names:
                counter += 1
            new = f"{new}_{counter}"

        used_names.add(new)
        final_mapping[original] = new

    # 컬럼명 변경
    df_renamed = df.rename(columns=final_mapping)

    print(f"컬럼명 변경 완료: {len(final_mapping)}개")
    print("변경된 주요 컬럼명:")
    for orig, new in list(final_mapping.items())[:10]:
        print(f"  {orig} -> {new}")

    return df_renamed, final_mapping


In [21]:
class FixedAdvancedEnsemble:
    """피처명 에러를 해결한 고성능 앙상블 모델"""

    def __init__(self):
        self.models = {}
        self.scalers = {}
        self.label_encoders = {}
        self.weights = None
        self.feature_mapping = None

    def prepare_features(self, df):
        """피처 준비 (컬럼명 정리 포함)"""

        # 컬럼명 정리
        df_clean, mapping = clean_feature_names(df)
        self.feature_mapping = mapping

        # 수치형/범주형 분리
        numeric_cols = df_clean.select_dtypes(include=[np.number]).columns.tolist()
        categorical_cols = df_clean.select_dtypes(include=['object']).columns.tolist()

        # 제거할 컬럼들
        remove_cols = ['target', 'target_log', 'is_test']
        numeric_cols = [col for col in numeric_cols if col not in remove_cols]

        return df_clean, numeric_cols, categorical_cols

    def encode_categorical(self, df, categorical_cols, fit=True):
        """범주형 변수 인코딩 (에러 처리 강화)"""

        df_encoded = df.copy()

        for col in categorical_cols:
            try:
                if fit:
                    le = LabelEncoder()
                    # 결측치 처리
                    df_encoded[col] = df_encoded[col].fillna('UNKNOWN')
                    # 모든 고유값을 문자열로 변환하여 fit
                    unique_vals = df_encoded[col].astype(str).unique()
                    le.fit(unique_vals)
                    self.label_encoders[col] = le
                else:
                    le = self.label_encoders[col]
                    # 결측치 처리
                    df_encoded[col] = df_encoded[col].fillna('UNKNOWN')

                    # 새로운 값들을 처리
                    for val in df_encoded[col].astype(str).unique():
                        if val not in le.classes_:
                            le.classes_ = np.append(le.classes_, val)

                df_encoded[col] = le.transform(df_encoded[col].astype(str))

            except Exception as e:
                print(f"Warning: {col} 인코딩 중 오류 발생: {e}")
                # 오류 발생 시 단순 숫자 인코딩
                df_encoded[col] = pd.factorize(df_encoded[col])[0]

        return df_encoded

    def get_models(self):
        """모델 정의 (에러 처리 강화)"""

        return {
            'lgb': lgb.LGBMRegressor(
                objective='regression',
                metric='rmse',
                boosting_type='gbdt',
                num_leaves=31,
                learning_rate=0.05,
                feature_fraction=0.9,
                bagging_fraction=0.8,
                bagging_freq=5,
                verbose=-1,
                random_state=42,
                force_col_wise=True,  # 메모리 효율성
                n_jobs=1  # 안정성을 위해 단일 스레드
            ),

            'xgb': xgb.XGBRegressor(
                objective='reg:squarederror',
                eval_metric='rmse',
                learning_rate=0.05,
                max_depth=6,
                subsample=0.8,
                colsample_bytree=0.8,
                random_state=42,
                n_jobs=1,
                verbosity=0
            ),

            'rf': RandomForestRegressor(
                n_estimators=50,  # 빠른 실행을 위해 줄임
                max_depth=15,
                min_samples_split=5,
                min_samples_leaf=2,
                random_state=42,
                n_jobs=1
            )
        }

    def train(self, X_train, y_train, X_val=None, y_val=None):
        """앙상블 모델 훈련 (에러 처리 강화)"""

        print("모델 훈련 중...")

        # 피처 준비
        X_train_clean, numeric_cols, categorical_cols = self.prepare_features(X_train)

        # 범주형 변수 인코딩
        X_train_encoded = self.encode_categorical(X_train_clean, categorical_cols, fit=True)

        if X_val is not None:
            X_val_clean, _, _ = clean_feature_names(X_val)
            X_val_encoded = self.encode_categorical(X_val_clean, categorical_cols, fit=False)

        # 모델 훈련
        models = self.get_models()
        val_predictions = {}

        for name, model in models.items():
            try:
                print(f"Training {name}...")

                # 피처 선택
                if name in ['lgb', 'xgb']:
                    # 트리 기반 모델: 모든 피처 사용
                    feature_cols = numeric_cols + categorical_cols
                    X_train_model = X_train_encoded[feature_cols]
                    X_val_model = X_val_encoded[feature_cols] if X_val is not None else None
                else:
                    # RandomForest: 수치형 피처만 사용 (안전)
                    X_train_model = X_train_encoded[numeric_cols]
                    X_val_model = X_val_encoded[numeric_cols] if X_val is not None else None

                # NaN 값 처리
                X_train_model = X_train_model.fillna(0)
                if X_val_model is not None:
                    X_val_model = X_val_model.fillna(0)

                # 모델 훈련
                model.fit(X_train_model, y_train)
                self.models[name] = model

                # 검증 예측
                if X_val is not None:
                    val_pred = model.predict(X_val_model)
                    val_predictions[name] = val_pred
                    rmse = np.sqrt(mean_squared_error(y_val, val_pred))
                    print(f"{name} Validation RMSE: {rmse:.2f}")

            except Exception as e:
                print(f"Error training {name}: {e}")
                continue

        # 가중치 계산
        if X_val is not None and val_predictions:
            rmse_scores = {}
            for name, pred in val_predictions.items():
                rmse_scores[name] = np.sqrt(mean_squared_error(y_val, pred))

            # RMSE 역수로 가중치 계산
            total_inv_rmse = sum(1/rmse for rmse in rmse_scores.values())
            self.weights = {name: (1/rmse)/total_inv_rmse for name, rmse in rmse_scores.items()}

            print("\n모델 가중치:")
            for name, weight in self.weights.items():
                print(f"{name}: {weight:.3f}")

    def predict(self, X_test):
        """앙상블 예측"""

        # 피처 준비
        X_test_clean, numeric_cols, categorical_cols = self.prepare_features(X_test)

        # 범주형 변수 인코딩
        X_test_encoded = self.encode_categorical(X_test_clean, categorical_cols, fit=False)

        # 각 모델 예측
        predictions = {}
        for name, model in self.models.items():
            try:
                if name in ['lgb', 'xgb']:
                    feature_cols = numeric_cols + categorical_cols
                    X_test_model = X_test_encoded[feature_cols]
                else:
                    X_test_model = X_test_encoded[numeric_cols]

                # NaN 값 처리
                X_test_model = X_test_model.fillna(0)

                predictions[name] = model.predict(X_test_model)

            except Exception as e:
                print(f"Error predicting with {name}: {e}")
                continue

        if not predictions:
            raise Exception("모든 모델에서 예측 실패")

        # 가중 평균
        if self.weights is not None:
            ensemble_pred = np.zeros(len(X_test))
            for name, pred in predictions.items():
                if name in self.weights:
                    ensemble_pred += pred * self.weights[name]
        else:
            # 단순 평균
            ensemble_pred = np.mean(list(predictions.values()), axis=0)

        return ensemble_pred

In [24]:
def fixed_main_pipeline(train_path, test_path):
    """에러 수정된 메인 실행 파이프라인"""

    print("=" * 60)
    print("수정된 서울 아파트 실거래가 예측 파이프라인")
    print("=" * 60)

    try:
        # 1. 데이터 로딩
        dt = pd.read_csv(train_path)
        dt_test = pd.read_csv(test_path)

        print(f"Train 데이터: {dt.shape}")
        print(f"Test 데이터: {dt_test.shape}")

        # 2. 기본 전처리
        dt['is_test'] = 0
        dt_test['is_test'] = 1
        dt_test['target'] = np.nan

        concat = pd.concat([dt, dt_test], ignore_index=True)

        # 3. 간단한 결측치 처리
        # 결측치 90% 이상인 컬럼 제거
        missing_ratio = concat.isnull().sum() / len(concat)
        high_missing_cols = missing_ratio[missing_ratio > 0.9].index.tolist()
        print(f"고결측률 컬럼 제거: {len(high_missing_cols)}개")

        concat = concat.drop(columns=high_missing_cols)

        # 수치형/범주형 결측치 처리
        for col in concat.columns:
            if concat[col].dtype in ['object']:
                concat[col] = concat[col].fillna('UNKNOWN')
            else:
                concat[col] = concat[col].fillna(concat[col].median())

        # 4. 기본 피처 엔지니어링
        if '시군구' in concat.columns:
            try:
                concat['구'] = concat['시군구'].str.split().str[1]
                concat['동'] = concat['시군구'].str.split().str[2]
            except:
                pass

        if '계약년월' in concat.columns:
            concat['계약년월'] = concat['계약년월'].astype(str)
            concat['계약년'] = concat['계약년월'].str[:4].astype(int)
            concat['계약월'] = concat['계약년월'].str[4:].astype(int)

        # 5. 데이터 분할
        train_data = concat[concat['is_test'] == 0].copy()
        test_data = concat[concat['is_test'] == 1].copy()

        # 타겟 변수 분리
        X = train_data.drop(['target', 'is_test'], axis=1, errors='ignore')
        y = train_data['target'] if 'target' in train_data.columns else None
        X_test = test_data.drop(['target', 'is_test'], axis=1, errors='ignore')

        if y is None:
            raise Exception("타겟 변수를 찾을 수 없습니다.")

        # 6. 훈련/검증 분할
        X_train, X_val, y_train, y_val = train_test_split(
            X, y, test_size=0.2, random_state=42
        )

        print(f"훈련 데이터: {X_train.shape}")
        print(f"검증 데이터: {X_val.shape}")

        # 7. 모델 훈련
        ensemble = FixedAdvancedEnsemble()
        ensemble.train(X_train, y_train, X_val, y_val)

        # 8. 최종 예측
        print("\n최종 예측 중...")
        predictions = ensemble.predict(X_test)

        # 9. 후처리
        predictions = np.maximum(predictions, 0)  # 음수 제거
        predictions = np.round(predictions).astype(int)  # 정수 변환

        # 10. 결과 저장
        result_df = pd.DataFrame({'target': predictions})
        result_df.to_csv('fixed_predictions.csv', index=False)

        print(f"예측 완료!")
        print(f"결과 저장: fixed_predictions.csv")
        print(f"예측값 범위: {predictions.min()} ~ {predictions.max()}만원")
        print(f"예측값 평균: {predictions.mean():.0f}만원")

        return predictions, ensemble

    except Exception as e:
        print(f"에러 발생: {e}")
        print("베이스라인 RandomForest로 대체 실행...")

        # 베이스라인 대체 실행
        return baseline_fallback(train_path, test_path)

def baseline_fallback(train_path, test_path):
    """에러 발생 시 베이스라인 대체"""

    from sklearn.ensemble import RandomForestRegressor
    from sklearn.preprocessing import LabelEncoder

    # 데이터 로딩
    dt = pd.read_csv(train_path)
    dt_test = pd.read_csv(test_path)

    # 간단한 전처리
    dt['is_test'] = 0
    dt_test['is_test'] = 1
    dt_test['target'] = np.nan

    concat = pd.concat([dt, dt_test], ignore_index=True)

    # 결측치 처리
    for col in concat.columns:
        if concat[col].dtype == 'object':
            concat[col] = concat[col].fillna('UNKNOWN')
        else:
            concat[col] = concat[col].fillna(concat[col].median())

    # 범주형 인코딩
    le_dict = {}
    for col in concat.select_dtypes(include=['object']).columns:
        le = LabelEncoder()
        concat[col] = le.fit_transform(concat[col].astype(str))
        le_dict[col] = le

    # 데이터 분할
    train_data = concat[concat['is_test'] == 0]
    test_data = concat[concat['is_test'] == 1]

    X = train_data.drop(['target', 'is_test'], axis=1)
    y = train_data['target']
    X_test = test_data.drop(['target', 'is_test'], axis=1)

    # 모델 훈련
    model = RandomForestRegressor(n_estimators=50, random_state=42, n_jobs=1)
    model.fit(X, y)

    # 예측
    predictions = model.predict(X_test)
    predictions = np.maximum(predictions, 0)
    predictions = np.round(predictions).astype(int)

    # 저장
    result_df = pd.DataFrame({'target': predictions})
    result_df.to_csv('baseline_predictions.csv', index=False)

    print("베이스라인 완료: baseline_predictions.csv")

    return predictions, model

# 실행
if __name__ == "__main__":
    train_path = '/home/data/train.csv'
    test_path = '/home/data/test.csv'

    predictions, model = fixed_main_pipeline(train_path, test_path)

수정된 서울 아파트 실거래가 예측 파이프라인
Train 데이터: (1118822, 52)
Test 데이터: (9272, 51)
고결측률 컬럼 제거: 4개
훈련 데이터: (895057, 51)
검증 데이터: (223765, 51)
모델 훈련 중...
컬럼명 정리 중...
컬럼명 변경 완료: 51개
변경된 주요 컬럼명:
  시군구 -> district
  번지 -> street_num
  본번 -> main_num
  부번 -> sub_num
  아파트명 -> apt_name
  전용면적(㎡) -> feature_5
  계약년월 -> contract_ym
  계약일 -> contract_day
  층 -> floor
  건축년도 -> build_year
컬럼명 정리 중...
컬럼명 변경 완료: 51개
변경된 주요 컬럼명:
  시군구 -> district
  번지 -> street_num
  본번 -> main_num
  부번 -> sub_num
  아파트명 -> apt_name
  전용면적(㎡) -> feature_5
  계약년월 -> contract_ym
  계약일 -> contract_day
  층 -> floor
  건축년도 -> build_year
에러 발생: not enough values to unpack (expected 3, got 2)
베이스라인 RandomForest로 대체 실행...
베이스라인 완료: baseline_predictions.csv
